In [ ]:
# 3. control sample rate;
# 4. mAP 


# 1. valid set 做infer upload
# 2. json 文件评估准确度；
# 3. 跟线上 map 比较；


In [1]:
import cv2
import os
import torch
import json
from copy import deepcopy
import ipdb
import numpy as np
from torchvision import models
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt
from data2 import Jiu2Data
from data import JiuData, do_resize, JiuTest, JiuValid
from model import *
from augment import *
from sampler import JiuSampler
from loss import multi_class_entropy, lovasz_softmax, FocalLoss2d, DiceLoss2D, FocalLoss2D_2
from fastprogress.fastprogress import master_bar, progress_bar
from eval import *
from timeit import default_timer as timer
from visdom import Visdom
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


def time_to_str(t, mode='min'):
    from timeit import default_timer as timer
    
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError


def train(loader, model, dataset, device):
    model = model.to(device)
    running_loss = 0.
    model.train()
    for imgs, masks, H, W in progress_bar(loader, parent=mb):
        optimizer.zero_grad()
#         ipdb.set_trace()
        imgs, masks = imgs.to(device), masks.to(device)
        with torch.set_grad_enabled(True):
            logits, cls_logits = model(imgs) # [bs, cls, H, W] [bs, cls]
#             loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss1 = FocalLoss2D_2(num_class=num_class, device=device)(logits, masks.squeeze().int())
#             loss2 = DiceLoss2D(cls_num=11, device=device)(logits, masks.squeeze().int())
#             loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
#             loss3 = torch.nn.CrossEntropyLoss()(cls_logits, cls)
            loss = loss1 
            print(loss1, logits.sum())
            loss.backward()
            optimizer.step()
        running_loss += loss.item()*imgs.size(0)
    return running_loss/len(dataset)


def valid(loader, model, dataset, device):
    model = model.to(device)
    run_loss, run_loss1, run_loss2, run_loss3 = 0, 0, 0, 0
    model.eval()
    for imgs, masks, H, W in progress_bar(loader, parent=mb):    
        imgs, masks = imgs.to(device), masks.to(device)
        with torch.no_grad():
            logits, cls_logits = model(imgs)
#             loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss1 = FocalLoss2D_2(num_class=num_class, device=device)(logits, masks.squeeze().int())
            
#             loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
#             loss2 = DiceLoss2D(cls_num=11, device=device)(logits, masks.squeeze().int())
#             loss3 = torch.nn.CrossEntropyLoss()(cls_logits, cls)
            
            loss = loss1
        run_loss += loss.item()*imgs.size(0)
#         run_loss1 += loss1.item()*imgs.size(0)
#         run_loss2 += loss2.item()*imgs.size(0)
#         run_loss3 += 0
    return run_loss/len(dataset), run_loss1/len(dataset), run_loss2/len(dataset), run_loss3/len(dataset)


def valid_look(model, fold_id, epoch, show_num=3, pth=1):
    # logit # bs, 11, h, w 
    # mask_pred
    # bbox_pred
    # picture
    if pth is None:
        param = torch.load('./models/unet_49' + '.pth')  # stage3 use model pretrained with pseudo-labels
        model.load_state_dict(param)  # initialize with pretained weight
    model = model.to(device)
    model.eval()
    valid_data_tmp = Jiu2Data(fold_id=fold_id, mode='valid', return_ori_img=True)
    loader = DataLoader(valid_data_tmp,
#                         sampler = RandomSampler(valid_data_tmp),
                        batch_size=1,
                        num_workers=8,
                        pin_memory=True)
    i = 0
    for imgs, mask, H, W, img_ori in loader:   
        if i < show_num:
            imgs = imgs.to(device)
            with torch.no_grad():
                mask_pred, cls_logit = model(imgs) # [num_cls, w, h]
                mask_pred = mask_pred.to('cpu').softmax(1).argmax(1)[0]
                bbox_pred = mask2bbox_withscale(mask_pred.numpy(), H, W)
                img  = img_ori[0].numpy() # h,w,3
                
                img_2 = deepcopy(img)
                x, y, w, h = [int(i) for i in bbox.numpy()[0]]
                x1, y1, w1, h1 = [int(i) for i in bbox_pred]
                print('mask pred rectangle: {}/{}'.format((x1, y1), (x1 + w1, y1 + h1)))
                print('mask real rectangle: {}/{}'.format((x, y), (x + w, y + h)))
                cv2.rectangle(img, (x, y), (x + w, y + h), (1, 0, 0), 2)
                cv2.rectangle(img_2, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 1), 2)
#                 ipdb.set_trace()
                import matplotlib.pyplot as plt
                tmp = np.array([0,1,2])+i*3
                plt.subplot(331 + tmp[0])
                plt.imshow(mask_pred)
                plt.title('mask_epoch:{}'.format(epoch))
                
                plt.subplot(331 + tmp[1])
                plt.imshow(img)
                plt.title('val_gt')
                
                plt.subplot(331 + tmp[2])
                plt.imshow(img_2)
                plt.title('val_pred_{}'.format(valid_data_tmp.id2cls[C.tolist()[0]]))
                
                i+=1
            vis.matplot(plt) 
        else:
            break
    return 


weight_dic = {1:0.15,
2:0.09,
3:0.09,
4:0.05,
5:0.13,
6:0.05,
7:0.12,
8:0.13,
9:0.07,
10:0.12}

In [2]:
batch_size = 20
num_class = 24
model = Unet_qb(num_class=num_class, HyperColumn=True)
lr = 0.01
min_lr = 0.005
EPOCH=200
snapshot = 3
scheduler_step = EPOCH//snapshot
device = 'cuda'
fold_id = 1
max_batchs_per_epoch = 300

train_data = Jiu2Data(fold_id=fold_id, mode='train')
train_loader = DataLoader(
                    train_data,
                    shuffle=RandomSampler(train_data),
#                     sampler=JiuSampler(train_data, bs=batch_size, max_batchs_per_epoch=max_batchs_per_epoch),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

valid_data = Jiu2Data(fold_id=fold_id, mode='valid')
valid_loader = DataLoader(
                    valid_data,
                    shuffle=RandomSampler(valid_data),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

img_path = './chongqing1_round1_train1_20191223/images/'
test_img_path = './chongqing1_round1_testA_20191223/images/'

testset = JiuTest(test_img_path=test_img_path)
test_loader = DataLoader(
                    testset,
                    batch_size=1,
                    num_workers=8,
                    pin_memory=True)

# validset = JiuValid(fold_id=fold_id)
# valid_infer_loader = DataLoader(
#                     validset,
#                     batch_size=1,
#                     num_workers=8,
#                     pin_memory=True)



In [3]:
# !python -m visdom.server

In [4]:
vis = Visdom(env='Friday1')

Setting up a new session...


In [5]:
optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
start = timer()
mb = master_bar(range(EPOCH))
for epoch in mb:
    
# DEBUG
# if 1:
#     PATH='./models/unet_qb_aug_focal'+ str(99) + '.pth'
#     model.load_state_dict(torch.load(PATH))
#     valid_look(model, fold_id, epoch=111)


#     training;
    scheduler.step()
    epoch_loss = train(train_loader, model, train_data, device)
    print(epoch_loss)
    print('-------')
    l_all, l1, l2, l3 = valid(valid_loader, model, valid_data, device)
    mAP = eval_mAP(loader=valid_infer_loader, model=model)
#     valid_look(model, fold_id, epoch)
    vis.line(X=[epoch], Y=[[scheduler.get_lr()[0], mAP, epoch_loss, l_all, l1, l2, l3]], opts=dict(markers=True, showlegend=True), win='loss', update='append' if epoch>0 else None)
    
#     重置lr;
    if (epoch + 1) % scheduler_step == 0:
        torch.save(model.state_dict(),  './models/unet_qb_aug_focal'+ str(epoch) + '.pth')
        optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
        scheduler.step()
        

tensor(4.8439, device='cuda:0', grad_fn=<MeanBackward0>) tensor(300820.4062, device='cuda:0', grad_fn=<SumBackward0>)
tensor(5.2347, device='cuda:0', grad_fn=<MeanBackward0>) tensor(173793.7812, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4.4053, device='cuda:0', grad_fn=<MeanBackward0>) tensor(36317.3555, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3.7915, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-166669.1719, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3.4061, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-517090.4688, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3.1423, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-1103831.3750, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.8275, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-1810875., device='cuda:0', grad_fn=<SumBackward0>)
tensor(2.1819, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-2685516., device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.8732, device='cuda:0', grad_fn=<MeanBackward0>) 

tensor(1.1921, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-94274.9062, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.6338, device='cuda:0', grad_fn=<MeanBackward0>) tensor(736916.1250, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.9539, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-497114.8750, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.3451, device='cuda:0', grad_fn=<MeanBackward0>) tensor(509993.0625, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.6516, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-399946.3438, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.3734, device='cuda:0', grad_fn=<MeanBackward0>) tensor(716395.5625, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.0405, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-848045.5000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.4440, device='cuda:0', grad_fn=<MeanBackward0>) tensor(127462.0312, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.8540, device='cuda:0', grad_fn=<MeanBackward

tensor(0.7243, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-411399.0938, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.3079, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-74997.5234, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.3257, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-410285.0312, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.8310, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-296551.3438, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.4918, device='cuda:0', grad_fn=<MeanBackward0>) tensor(384215.3125, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.4845, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-290120.3438, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.8597, device='cuda:0', grad_fn=<MeanBackward0>) tensor(-219330.1562, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.4613, device='cuda:0', grad_fn=<MeanBackward0>) tensor(642346.6875, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.0858, device='cuda:0', grad_fn=<MeanBackwa

NameError: name 'valid_infer_loader' is not defined

In [8]:
# def lr_exp_list(min_lr=0.0001, max_lr=3, iters=300, mode='exp'):
#     assert mode in ['exp', 'linear']
#     if mode=='exp':
#         lr_list = list(reversed([max_lr*0.915**i for i in range(iters)]))
#     else:
#         step = (lr_max-lr_min)/iters
#         lr_list = [step*i for i in range(1,iters+1)]
#     return lr_list

# def lr_range_test(loader, model, dataset, device, min_lr, iters=300):
#     '''
#     try to find best lr range;
#     return batch_loss_list;
#     '''
#     batch_loss_list = []
#     model = model.to(device)
#     running_loss = 0.
#     model.train()
#     iter_id = 0 
#     lr_list = lr_exp_list(min_lr=min_lr, iters=iters)
#     vis = Visdom(env = 'lr_range_test1')
#     for imgs, masks, H, W in progress_bar(loader):
#         if iter_id <= iters:
#             lr_tmp = lr_list[iter_id]
#             iter_id += 1
#             optimizer = torch.optim.SGD(params = model.parameters(), lr=lr_tmp)
#             optimizer.zero_grad()
#             imgs, masks = imgs.to(device), masks.to(device)
#             with torch.set_grad_enabled(True):
#                 logits, cls_logits = model(imgs) # [bs, cls, H, W] [bs, cls]
#     #             loss1 = multi_class_entropy(logits, masks.squeeze().int())
#                 loss1 = FocalLoss2D_2(num_class=num_class, device=device)(logits, masks.squeeze().int())
#     #             loss2 = DiceLoss2D(cls_num=11, device=device)(logits, masks.squeeze().int())
#     #             loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
#     #             loss3 = torch.nn.CrossEntropyLoss()(cls_logits, cls)
#                 loss = loss1 
#                 loss.backward()
#                 optimizer.step()
#             print(lr_tmp, loss.item())
#             batch_loss_list.append([lr_tmp, loss.item()])
#             vis.line(X=[lr_tmp], Y=[[loss.item()]], opts=dict(markers=True, showlegend=True), win='loss', update='append')
#         else:
#             break
#     return batch_loss_list

# if __name__ == '__main__':
#     from torch.utils.data import DataLoader
#     from torch.utils.data.sampler import RandomSampler
#     from loss import multi_class_entropy, lovasz_softmax, FocalLoss2d, DiceLoss2D, FocalLoss2D_2
#     from data2 import Jiu2Data
#     from model import *
#     from loss import multi_class_entropy, lovasz_softmax
#     from fastprogress.fastprogress import master_bar, progress_bar
#     from visdom import Visdom

#     num_class = 24
#     batch_size = 20
#     model = Unet_qb(num_class=num_class, HyperColumn=True)

#     device = 'cuda'
#     fold_id = 1

#     train_data = Jiu2Data(fold_id=fold_id, mode='train')
#     train_loader = DataLoader(
#                         train_data,
#                         shuffle=RandomSampler(train_data),
#                         batch_size=batch_size,
#                         num_workers=8,
#                         pin_memory=True)

#     res = lr_range_test(train_loader, model, train_data, device, min_lr=0.0001, iters=170)
#     # [0.005, 0.01]


Setting up a new session...


/home/qibo/qb_vir_env/lib/python3.6/site-packages/torch/nn/functional.py:2457: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


9.063295749871587e-07 4.650514602661133
9.905241256690258e-07 4.835926055908203
1.08254002805358e-06 5.6113972663879395
1.1831038557962623e-06 4.589935302734375
1.2930096784658604e-06 4.1842780113220215
1.413125331656678e-06 4.372176647186279
1.5443992695701398e-06 5.557719707489014
1.6878680541750162e-06 4.795699596405029
1.8446645400819847e-06 4.770511150360107
2.016026819761732e-06 4.739486217498779
2.2033079997395975e-06 5.322046756744385
2.4079868849613086e-06 4.957401275634766
2.6316796556954193e-06 5.6952805519104
2.876152629175321e-06 4.8528666496276855
3.143336206748985e-06 4.703388214111328
3.435340116665557e-06 5.017614841461182
3.7544700728585323e-06 4.6367082595825195
4.103245981266156e-06 4.908940315246582
4.48442183744935e-06 4.403446197509766
4.901007472622241e-06 4.683043003082275
5.356292319805727e-06 4.361291885375977
5.853871387765822e-06 5.71571159362793
6.397673647831499e-06 4.436385631561279
6.991993057739342e-06 4.415324687957764
7.64152246747469e-06 4.861566543

IndexError: list index out of range